In [1]:
import pandas as pd
import numpy as np

In [ ]:
train = pd.read_csv('../preprocessed_data/kneighbors_05_06_2025_train.csv')
test = pd.read_csv('../preprocessed_data/kneighbors_05_06_2025_test.csv')

train.head()

C:\Users\irela\AppData\Local\Temp\ipykernel_14616\3967878826.py:1: DtypeWarning: Columns (2,16) have mixed types. Specify dtype option on import or set low_memory=False.
  train = pd.read_csv('../preprocessed_data/kneighbors_05_06_2025_train.csv')


,EventTimeStamp,spn,EquipmentID,BarometricPressure,DistanceLtd,EngineCoolantTemperature,EngineLoad,EngineOilPressure,EngineOilTemperature,EngineRpm,...,fmi_23,fmi_29,fmi_3,fmi_31,fmi_4,fmi_5,fmi_6,fmi_7,fmi_8,fmi_9
0,2018-07-20 09:31:33,1231,105406655,-0.045593,-1.915080,0.361517,-0.649222,0.533916,0.267595,0.383988,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,2018-07-20 09:42:30,1231,105406655,-7.142936,-0.061030,-0.864916,-0.021608,0.420676,0.224204,-0.439622,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,2016-01-31 07:12:25,629,105301976,0.112396,-2.211322,-0.142197,-0.583845,-2.955934,-0.153681,-2.136658,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,2018-01-26 18:58:08,37,105338729,0.318997,-2.546049,0.033008,-0.518469,-1.113211,0.214502,-0.962471,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,2018-01-26 18:59:36,37,105338729,0.161008,-2.155977,0.153461,0.109145,-0.248470,-0.743031,-0.842814,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [11]:
train[train['FullDerate'] == 1].head()
# If Derate occured two hours before the current derate, remove the row

# Convert to datetime if needed
train['EventTimeStamp'] = pd.to_datetime(train['EventTimeStamp'])

# Filter derates and sort by time
derates = train[train['FullDerate'] == 1].sort_values('EventTimeStamp').copy()

# Initialize list to keep valid derates
kept_indices = []

# Track timestamp of last kept derate
last_kept_time = None

for idx, row in derates.iterrows():
    if last_kept_time is None or (row['EventTimeStamp'] - last_kept_time).total_seconds() > 7200:
        kept_indices.append(idx)
        last_kept_time = row['EventTimeStamp']

# Rebuild the filtered derate DataFrame
derates_filtered = derates.loc[kept_indices]

# Combine with non-derate rows
train_filtered = pd.concat([
    train[train['FullDerate'] != 1],
    derates_filtered
]).sort_values('EventTimeStamp').reset_index(drop=True)

len(train_filtered), len(train)


(938258, 938258)

In [12]:
test[test['FullDerate'] == 1].head()
# If Derate occured two hours before the current derate, remove the row

# Convert to datetime if needed
test['EventTimeStamp'] = pd.to_datetime(test['EventTimeStamp'])

# Filter derates and sort by time
derates = test[test['FullDerate'] == 1].sort_values('EventTimeStamp').copy()

# Initialize list to keep valid derates
kept_indices = []

# Track timestamp of last kept derate
last_kept_time = None

for idx, row in derates.iterrows():
    if last_kept_time is None or (row['EventTimeStamp'] - last_kept_time).total_seconds() > 7200:
        kept_indices.append(idx)
        last_kept_time = row['EventTimeStamp']

# Rebuild the filtered derate DataFrame
derates_filtered = derates.loc[kept_indices]

# Combine with non-derate rows
test_filtered = pd.concat([
    test[test['FullDerate'] != 1],
    derates_filtered
]).sort_values('EventTimeStamp').reset_index(drop=True)

len(test_filtered), len(test)


(110596, 110596)

In [5]:
#train_filtered.to_csv('../preprocessed_data/filtered_attempting_to_save_knimputer_05_03_train.csv', index=False)
#test_filtered.to_csv('../preprocessed_data/filtered_attempting_to_save_knimputer_05_03_test.csv', index=False)

In [ ]:
target_columns = ['HoursUntilNextDerate', 'DerateInNextTwoHours', 'DerateInNextTwentyFourHours']
y_train = train_filtered[target_columns].copy()
y_test = test_filtered[target_columns].copy()
y_train.head()

train_filtered = train_filtered.drop(columns=['EventTimeStamp', 'EquipmentID', 'NextDerateTime', 'HoursUntilNextDerate', 'FullDerate', 'DerateInNextTwoHours', 'DerateInNextTwentyFourHours'])
test_filtered = test_filtered.drop(columns=['EventTimeStamp', 'EquipmentID', 'NextDerateTime', 'HoursUntilNextDerate', 'FullDerate', 'DerateInNextTwoHours', 'DerateInNextTwentyFourHours'])

for target in target_columns:
    print(f"Processing target: {target}")
    new_train_df = pd.concat([y_train[target], train_filtered], axis=1)
    new_test_df = pd.concat([y_test[target], test_filtered], axis=1)
    new_train_df.to_csv(f'../preprocessed_data/{target}_kneighbors_05_06_train.csv', index=False)
    new_test_df.to_csv(f'../preprocessed_data/{target}_kneighbors_05_06_test.csv', index=False)
    new_train_df = pd.DataFrame()
    new_test_df = pd.DataFrame()
    print(f"Saved {target} dataframes for training and testing.")

Processing target: HoursUntilNextDerate
Saved HoursUntilNextDerate dataframes for training and testing.
Processing target: DerateInNextTwoHours
Saved DerateInNextTwoHours dataframes for training and testing.
Processing target: DerateInNextTwentyFourHours
Saved DerateInNextTwentyFourHours dataframes for training and testing.


In [7]:
new_train_df.head()

""


In [8]:
train_filtered.head()

,spn,BarometricPressure,DistanceLtd,EngineCoolantTemperature,EngineLoad,EngineOilPressure,EngineOilTemperature,EngineRpm,FuelLtd,FuelRate,...,fmi_23,fmi_29,fmi_3,fmi_31,fmi_4,fmi_5,fmi_6,fmi_7,fmi_8,fmi_9
0,829,0.197467,-0.629948,0.416268,-0.551157,-0.701429,0.057964,-0.976154,-0.524722,-0.681251,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,96,0.197467,-0.629948,0.416268,-0.551157,-0.701429,0.057964,-0.976154,-0.524722,-0.681251,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,96,0.197467,-0.629948,0.416268,-0.551157,-0.701429,0.057964,-0.976154,-0.524722,-0.681251,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,829,0.197467,-0.629948,0.416268,-0.551157,-0.701429,0.057964,-0.976154,-0.524722,-0.681251,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,792,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0


In [9]:
#my_train = pd.read_csv('../preprocessed_data/model_filtered_attempting_to_save_knimputer_05_03_train.csv')
#my_test = pd.read_csv('../preprocessed_data/model_filtered_attempting_to_save_knimputer_05_03_test.csv')

#my_train.head()